# Problem Set 3: Diagnosis & Conflict-directed A*
## Due Wednesday, March 15, 11:59 p.m.

In this problem set, you'll implement conflict-directed A* (CD-A*) and use it to solve mode estimation problems for model-based diagnosis. Specifically, you'll apply it to the Boolean polycell example from lecture.

<div class="alert alert-warning">
<strong>Note:</strong> We have provided a tutorial in the same folder as this notebook.
</div>

1. [Modeling](#modeling)
  1. [Modeling example](#modeling-example)
  2. [Modeling API](#modeling-api)
  3. [Modeling Boolean polycell (30 pts)](#modeling-boolean-polycell)
2. [Implementing Conflict-directed A*](#implementing-cdastar)
  1. [Estimating probabilities of partial assignments](#estimating-probability)
  2. [Manifesting & resolving conflicts (20 pts)](#conflicts)
  3. [Splitting on variables & conflicts (20 pts)](#splitting)
  4. [Search algorithm](#search-algorithm)
  5. [Testing CD-A* (30 pts)](#testing)

Make sure you load the dependencies below by highlighting the cell below and pressing Shift + Enter.

In [4]:
%load_ext autoreload
%autoreload 2
from propositional_state_logic import *
from sat_solver import *
from utils import *

## Modeling <a id="modeling"/>

We've given you some tools to build on to make your implementation easier. Namely, we've implemented a general propositional state logic system, where you can declare variables, add arbitrary propositional constraints, check satisfiability (our SAT solver uses DPLL with unit propagation if you're curious), and automatically extract conflicts when the problem isn't satisfiable. You won't have to implement any of these since we're providing them for you. Feel free to check out the code if you're curious how they work though.

### Modeling example <a id="modeling-example"/>

Let's dive right in to modeling; consider the following logical word problem:

> You're on a sailboat. The weather can be sunny, gusty, or rainy. Your boat can be anchored or not. The ocean can be idle or have a current. The sailboat can be stopped or moving. If the boat is anchored, it won't be moving. Otherwise, it definitely will be moving if there's ocean current or the weather is gusty. You're trying to decide where to stand on your boat: If it's moving, you don't want to be inside in your windowless cabin because you'll get seasick. If it's rainy, you don't want to stand outside on deck because you'll get soaked.

Let's model this problem with the API we provide:

In [2]:
p = Problem()

# Define the variables above. Returns a Variable object.
weather = p.add_variable('weather', type='finite_domain', domain=['sunny', 'gusty', 'rainy'])
anchored = p.add_variable('anchored', type='binary') # Binary variables have domain 'True' or 'False'.
ocean = p.add_variable('ocean', type='finite_domain', domain=['idle', 'current'])
speed = p.add_variable('speed', type='finite_domain', domain=['stopped', 'moving'])
standing = p.add_variable('standing', type='finite_domain', domain=['ondeck', 'inside'], decision_variable=True)

# Add the word problem constraints.
p.add_constraint('anchored => speed=stopped')
p.add_constraint('~anchored & (ocean=current | weather=gusty) => speed=moving')
p.add_constraint('~(standing=ondeck & weather=rainy)')
p.add_constraint('~(standing=inside & speed=moving)')

# Prints out constraints nicely in LaTeX, so you can check them.
display_constraints(p)

Constraints:


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

Great, we've modeled our problem. Now we can ask interesting questions. Suppose it's gusty out, but you want to stand inside. Is that even possible? Let's check:

In [3]:
sat = SATSolver(p)
# Make an assignment to some variables; variable.get_assignment(value) returns an Assignment.
assignment = frozenset([weather.get_assignment('gusty'), standing.get_assignment('inside')])
sat.check_consistency(assignment)

(True,
 frozenset({(ocean = current),
            (speed = stopped),
            (standing = inside),
            (weather = gusty),
            anchored}),
 None)

Yes, it is! Even though it's windy, the boat won't be moving (and hence you won't get seasick) if you keep the anchor down. The first returned value of `sat.check_consistency(...)` is `True` if the problem is satisfiable, `False` otherwise. The second returned value is a full assignment to all variables in your problem. The final returned value (`None` here) is a conflict, if it's not consistent.

Let's ask another question. You want to stand outside on deck. The weather is neither sunny nor gusty. Is that possible?

In [4]:
pn = p.copy() # Make an additional copy so we can add another constraint.
pn.add_constraint('~(weather=sunny | weather=gusty)')
pn.add_constraint('standing=ondeck')
sat = SATSolver(pn)
sat.check_consistency(frozenset())

(False, None, frozenset({(standing = ondeck)}))

No, that's not possible! Intuitively that makes sense; the weather must be rainy, and you can't stand outside while it's rainy. The first return value, `False`, indicates that this problem is not satisfiable. Hence the middle return value (assignments) is None. The final return value is a conflict, namely, a set of decision variables that cannot all hold true. We only have one decision variable in this toy example, but we correctly extract the conflict that we can't stand on deck.

### Modeling API <a id="modeling-api"/>

You can use the following operators, in addition to parentheses `(` and `)`, to build up your constraints:

| Name | Logical notation | Operator to use in formulas 
|:---:|:---:|:---:|
| And | $$ a_1 \wedge a_2 \wedge ... a_k $$ | `a_1 & a_2 & ... & a_k` |
| Or | $$ a_1 \vee a_2 \vee ... \vee a_k $$ | `a_1 `&#124;`a_2`&#124;`...`&#124;`a_k`  |
| Not | $$ \neg a $$ | ~x |
| Xor | $$ a_1 \oplus a_2 \oplus ... \oplus a_k $$ | `a_1 ^ a_2 ^ ... a^k`  |
| Implication | $$ x \Rightarrow y $$ | `x => y` |
| Equivalence | $$ x \Leftrightarrow y $$ | `x <=> y` |

You likely noticed you have to provide some information when you declare variables. A variable's `type` can be `'binary'` or `'finite_domain'`. `'binary'` variables automatically have a domain of `True` or `False`. `'finite_domain'` variables have a custom domain that you specify as a list. (Internally in the tools, they get compiled down to binary variables, one per domain value, representing whether the variable takes that value. We also add some additional constraints to make sure that exactly one of those values holds at a time.)

Notice that when we declared the `standing` variable, we also marked it as a `decision_variable`. All this is does is mark the variable as one over which we'll require a utility function to be defined (in our case, that utility is probability). These decision variables are what conflict-directed A* will be searching over. Our SAT-based consistency checker will search over all remaining variables during consistency checking. In this problem set, mode variables will be our decision variables. Note that any returned conflicts only involve decision variables.

We'll be representing partial assignemnts as `frozenset`s of Assignment objects (you may wish to review `set`s and `frozenset`s in Python). Briefly, a `frozenset` behaves like a normal `set`, except that it is immutable so you can't add or remove elements from it once it's created. This allows `frozenset`s to be hashable, so you can use them inside other `set`s or `frozenset`s and quickly check for membership.

Here are some other API functions you may find useful:

- `variable.domain` - A `frozenset` of values in this variable's domain.
- `variable.get_assignment(di)` - Returns an Assignment object assigning variable = di. An assignment object is just a single assignment, like $x = 1$.
- `assignment.var` - Returns the variable associated with the Assignment object.
- `assignment.val` - Returns the value associated with the Assignment object.
- `assignment.prob` - Returns the probability associated with this Assignment, if it's been set, else None. (You'll see how to set probabilities later in this notebook.)
- `prob.get_decision_variables()` - Returns a frozenset of all variables add to this problem that were marked as decision variables.

For more details, please refer to the Python files included at the top of this notebook.

### Modeling Boolean polycell (30 points) <a id="modeling-boolean-polycell"/>

First, we'll model the Boolean polycell example presented in lecture. We'll use propositional state logic to represent the operation of the system.

Recall the Boolean polycell example:
<img src="polycell.svg">

Below, we set up a problem and associated variables. Later, we'll add probabilities as well as observations such as `A=True`.

In [50]:
polycell = Problem()

# Variables
A1 = polycell.add_variable('A1', type='finite_domain', domain=['G', 'U'], decision_variable=True)
A2 = polycell.add_variable('A2', type='finite_domain', domain=['G', 'U'], decision_variable=True)
A3 = polycell.add_variable('A3', type='finite_domain', domain=['G', 'U'], decision_variable=True)
X1 = polycell.add_variable('X1', type='finite_domain', domain=['G', 'U'], decision_variable=True)
X2 = polycell.add_variable('X2', type='finite_domain', domain=['G', 'U'], decision_variable=True)
A = polycell.add_variable('A', type='binary')
B = polycell.add_variable('B', type='binary')
C = polycell.add_variable('C', type='binary')
D = polycell.add_variable('D', type='binary')
E = polycell.add_variable('E', type='binary')
F = polycell.add_variable('F', type='binary')
G = polycell.add_variable('G', type='binary')
X = polycell.add_variable('X', type='binary')
Y = polycell.add_variable('Y', type='binary')
Z = polycell.add_variable('Z', type='binary')

Enter your model constraints here:

<div class="alert alert-info">
Add your model constraints below. Don't add any "observation" constraints yet (e.g., like A = True, etc.); just model constraints about the polycell components' behavior. We'll add the observations later.
</div>

In [51]:
polycell.clear_constraints()

# polycell.add_constraint(' ... ')
# ...

# YOUR CODE HERE
polycell.add_constraint('~(A & C <=> X) => A1=U')
polycell.add_constraint('~(B & D <=> Y) => A2=U')
polycell.add_constraint('~(C & E <=> Z) => A3=U')
polycell.add_constraint('~(X ^ Y <=> F) => X1=U')
polycell.add_constraint('~(Y ^ Z <=> G) => X2=U')

display_constraints(polycell)

Constraints:


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

You can check your model here. This code will run a few tests against your model (but it isn't exhaustive!).

In [52]:
check_polycell_problem(polycell)
test_ok()

Checking mode and observation variables...
Checking A1...
Checking A2...
Checking A3...
Checking X1...
Checking X2...


Tests passed!!

## Implement Conflict-directed A* <a id="implementing-cdastar"/>

In this section, you'll implement conflict-directed A*. Please note that the _tutorial_ distributed with this notebook may help if you're unclear about how the algorithm works.

The following exercises will ask you to implement various functions, culminating in conflict-directed A*. Please define all functions below, and feel free to define any additional auxiliary functions if you want.

### Estimating probabilities of partial assignments <a id="estimating-probability">

Here, we return an optimistic estimate of the partial assignment's probability. This should both take into account the assigned variables, as well as all of the unassigned variables.

In [53]:
def get_max_probability_value(variable):
    return max(variable.get_assignment(di).prob for di in variable.domain)

def estimate_assignment_probability(assignments, decision_variables):
    """Optimistically estimates the probability of this assignment.
    
    Input: assignments - A frozenset of Assignment objects representing a partial assignment 
                         to decision variables.
           decision_variables - A frozenset of all of the problems' decision variables.
    Output: An admissible/optimistic estimate of the probability of of assignments.
    """
    p_a = 1.0
    # Probabilty of assignments
    for assignment in assignments:
        p_a *= assignment.prob
    # Unassigned variables: choose best unassigned heuristic
    for var in decision_variables - set(assignment.var for assignment in assignments):
        p_a *= get_max_probability_value(var)
    return p_a

### Manifesting & resolving conflicts (20 pts) <a id="conflicts"/>

Next, please implement functions for checking whether partial assignments manifest (given to you) or resolve conflicts.

<div class="alert alert-info">
Complete the functions below.
</div>

In [54]:
# Any auxiliary functions:
# YOUR CODE HERE

def manifests_conflict(assignments, conflict):
    """Checks to see if assignments manifests the conflict.

    Input: assignments - A frozenset of Assignment objects representing a partial assignment 
                         to decision variables.
           conflict - A frozenset of Assignment objects representing a conflict.
    Output: True or False.
    """
    return conflict.issubset(assignments)

def resolves_conflict(assignments, conflict):
    """Checks to see if assignments resolves the conflict.

    Input: assignments - A frozenset of Assignment objects representing a partial assignment 
                         to decision variables.
           conflict - A frozenset of Assignment objects representing a conflict.
    Output: True or False.
    """
    # YOUR CODE HERE
    conflictExists = manifests_conflict(assignments, conflict)
    return (not conflictExists)
    

In [55]:
def resolves_conflict(assignments, conflict):
    """Checks to see if assignments resolves the conflict.

    Input: assignments - A frozenset of Assignment objects representing a partial assignment 
                         to decision variables.
           conflict - A frozenset of Assignment objects representing a conflict.
    Output: True or False.
    """
    all_resolving_assignments=set()
    for i in conflict:
        conflicted_var = i.var
        conflicted_var_domain = i.var.domain
        conflicted_assignment = i.val
        resolving_assignments = set(conflicted_var.get_assignment(j) for j in conflicted_var_domain if j != conflicted_assignment)
        all_resolving_assignments = all_resolving_assignments.union(resolving_assignments)
    for resolving in all_resolving_assignments:
        if resolving in assignments:
            return True
    return False

You can test your function with the following code.

In [56]:
check_resolves_conflict(resolves_conflict)
test_ok()

Tests passed!!

### Splitting on variables & conflicts (20 pts) <a id="splitting"/>

Please implement the `split_on_conflict` function below. The other functions are given to you, and might be useful later for the search algorithm.

<div class="alert alert-info">
Complete the functions below.
</div>

In [57]:
# Any auxiliary functions:
def generate_constituent_kernels(conflict):
    """Apply De Morgan's Law and generate the constituent kernels
    
       Input: conflict - A frozenset of Assignment objects representing a conflict
       
       Output: a constituent kernel
    """
    constituent_kernel=[]
    
    for ci in conflict:
        
        #get all possible assignments for var of ci
        for value in ci.var.domain:
            
            #if assignment not conflict, append to kernel
            if value != ci.val:
                constituent_kernel.append(frozenset([ci.var.get_assignment(value)]))
    
    return constituent_kernel
                

def choose_variable_to_split_on(assignments, decision_variables):
    """Choose an unassigned variable to split on.

       Input: assignments - A frozenset of Assignment objects.
              decision_variables - A frozenset of all the decision variables.
    """
    vars_unassigned = decision_variables - frozenset(assignment.var for assignment in assignments)
    return list(vars_unassigned)[0]

def split_on_variable(assignments, variable):
    """Split on variable, returning a list of neighboring states.
    
    Input: assignments - A frozenset of Assignment objects representing a partial assignment.
           variable - A variable, not represented by an Assignment in assignments, on 
                      which to split via its domain values.
    
    Output: A list of frozensets of Assignment objects, representing a list of neighboring
            states.
    """
    neighbors = [(assignments | frozenset([variable.get_assignment(di)])) for di in variable.domain]
    return neighbors
    
def split_on_conflict(assignments, conflict):
    """Split on conflict, returning a list of neighboring states.
    
    Input: assignments - A frozenset of Assignment objects representing a partial assignment.
           conflict - A frozenset of Assignment objects representing a conflict.
    
    Output: A list of frozensets of Assignment objects, representing a list of neighboring
            states.
    """
    
    constituent_kernels = generate_constituent_kernels(conflict)
    
    neighbors = [assignments.union(ci) for ci in constituent_kernels]
    
    return neighbors

Test below.

In [58]:
check_split_on_conflict(split_on_conflict)
test_ok()

Tests passed!!

### Search algorithm <a id="search-algorithm"/>

Now that you've implemented a number of useful helper functions, please use them to implement the main conflict-directed A* search algorithm below. We have written up most of the code, so you only need to fill in the missing logic.

<div class="alert alert-info">
Complete the function below.
</div>

In [71]:
import heapq

# Any auxiliary functions here:
def add_to_priority_queue(Q, assignments, decision_variables):
    # Calculate f(x) = g(x) + h(x), where h is an admissible
    # heuristic for this assignment.
    p = estimate_assignment_probability(assignments, decision_variables)
    # Add to max priority queue (so negate).
    heapq.heappush(Q, (-p, assignments))
    
def is_complete_assignment(assignments, decision_variables):
    return decision_variables == set(assignment.var for assignment in assignments)

def conflict_directed_a_star(prob):
    """Main CD-A* function.

    Input: prob - A problem object like you declared above.
    Output: An assignment to all decision variables, represented as a frozenset of
            Assignment objects (note that only decision variables should be assigned).
            The returned result should be an optimal solution to the CSP. If no solution
            exists, returns None.
    """
    # Set up.
    decision_variables = prob.get_decision_variables()
    sat_solver = SATSolver(prob)
    Q = []
    Gamma = []
    expanded = set()
    add_to_priority_queue(Q, frozenset(), decision_variables)
    print(Q)
    while len(Q) > 0:
        # Pop Q and and the assignment to the expanded list.
        p, assignment = heapq.heappop(Q)
        print("Popped {} with p={}.".format(assignment, -p))
        expanded.add(assignment)
        # Remove any identical assignments from Q (optional).
        Q = [(p, a) for (p, a) in Q if a != assignment]

        if is_complete_assignment(assignment, decision_variables):
            print(" --> Checking consistency.")
            consistent, a, conflict = sat_solver.check_consistency(assignment)
            if consistent:
                print(" --> Yep! Done.")
                return assignment # Could also use yield to make a generator and keep enumerating!
            else:
                print(" --> Nope! Got conflict {}.".format(conflict))
                # Learn this conflict! Add it to Gamma if no gamma is already more general than
                # the conflict. Also discard any gammas that the conflict is more general than.
                if any(gamma.issubset(conflict) for gamma in Gamma):
                    continue
                Gamma = [gamma for gamma in Gamma if not conflict.issubset(gamma)]
                Gamma.append(conflict)
                # Remove assignments from Q manifesting this conflict.
                Q = [(p, a) for (p, a) in Q if not manifests_conflict(a, conflict)]
                heapq.heapify(Q)

        else:
            # Assignment is not a full assignment.

            # Implement the logic to either split on an unassigned variable,
            # or split on a conflict. Note that if there are unresolved conflicts,
            # you'll need to split on one of them first.
            # The neighbors created in each case will be added to Q later.
            # Remember: You have implemented these functions:
            #     - resolves_conflict
            #     - choose_variable_to_split_on (given to you)
            #     - split_on_variable (given to you)
            #     - split_on_conflict
            # If you read the other parts of the code, you'll see that Gamma is our set of conflicts.

            # YOUR CODE HERE
            conflict_exists = False
            
            for conflict in Gamma:
                if manifests_conflict(assignment, conflict):
                    conflict_exists = True
                    neighbors = split_on_conflict(assignment, conflict)
            
            if not conflict_exists: 
                xi = choose_variable_to_split_on(assignment, decision_variables)
                neighbors = split_on_variable(assignment, xi)

            # Add neighbors to the Q.
            for neigh in neighbors:
                if not neigh in expanded:
                    print(" --> Adding {}".format(neigh))
                    add_to_priority_queue(Q, neigh, decision_variables)

    # If we get here, no assignments found!
    return None

### Testing CD-A* (30 pts) <a id="testing"/>

You're done implementing conflict-directed A*! Time to test it. Let's also add some observation constraints:

In [72]:
polycell_obs = polycell.copy()
polycell_obs.add_constraint('A = True')
polycell_obs.add_constraint('B = True')
polycell_obs.add_constraint('C = True')
polycell_obs.add_constraint('D = False')
polycell_obs.add_constraint('E = True')
polycell_obs.add_constraint('F = False')
polycell_obs.add_constraint('G = True')

display_constraints(polycell_obs)

Constraints:


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

Next, we can specify the utility of the decision variables (i.e., the probability of various modes). Remember that we're making an independence assumption here. Then, run conflict-directed A*!

In [73]:
A1.set_probabilities({
    'G': 0.96,
    'U': 0.04,
})
A2.set_probabilities({
    'G': 0.95,
    'U': 0.05,
})
A3.set_probabilities({
    'G': 0.97,
    'U': 0.03,
})
X1.set_probabilities({
    'G': 0.98,
    'U': 0.02,
})
X2.set_probabilities({
    'G': 0.99,
    'U': 0.01,
})
polycell.check_probabilities() # Make sure they sum to 1.

True

And now we can check the solution:

In [74]:
# Run conflict-directed A*!
solution = conflict_directed_a_star(polycell_obs)
print("Solution: {}".format(solution))

# Is it the right solution?
assert_equal(solution, frozenset([A1.get_assignment('U'), A2.get_assignment('G'), A3.get_assignment('G'), X1.get_assignment('G'), X2.get_assignment('G')]))
test_ok()

[(-0.8582777279999999, frozenset())]
Popped frozenset() with p=0.8582777279999999.
 --> Adding frozenset({(A3 = U)})
 --> Adding frozenset({(A3 = G)})
Popped frozenset({(A3 = G)}) with p=0.8582777279999999.
 --> Adding frozenset({(A3 = G), (X1 = U)})
 --> Adding frozenset({(A3 = G), (X1 = G)})
Popped frozenset({(A3 = G), (X1 = G)}) with p=0.8582777279999999.
 --> Adding frozenset({(A1 = U), (A3 = G), (X1 = G)})
 --> Adding frozenset({(A3 = G), (X1 = G), (A1 = G)})
Popped frozenset({(A3 = G), (X1 = G), (A1 = G)}) with p=0.8582777279999999.
 --> Adding frozenset({(A3 = G), (X1 = G), (A2 = U), (A1 = G)})
 --> Adding frozenset({(A3 = G), (X1 = G), (A2 = G), (A1 = G)})
Popped frozenset({(A3 = G), (X1 = G), (A2 = G), (A1 = G)}) with p=0.8582777279999999.
 --> Adding frozenset({(A3 = G), (A2 = G), (X1 = G), (X2 = U), (A1 = G)})
 --> Adding frozenset({(A3 = G), (A2 = G), (X2 = G), (X1 = G), (A1 = G)})
Popped frozenset({(A3 = G), (A2 = G), (X2 = G), (X1 = G), (A1 = G)}) with p=0.858277727999999

Tests passed!!

In the above example, the most likely single fault was `A2 = U`. However, there would be no assignment to `X`, `Y`, and `Z` consistent with that and the given inputs and outputs. The next most likely single fault would be `A1 = U`, and it turns out this is a consistent diagnosis, so that's the result we expect from CD-A*. Now, let's change the testing example so that `X1 = U` is more likely.

In [75]:
A1.set_probabilities({
    'G': 0.99,
    'U': 0.01,
})
A2.set_probabilities({
    'G': 0.96,
    'U': 0.04,
})
A3.set_probabilities({
    'G': 0.97,
    'U': 0.03,
})
X1.set_probabilities({
    'G': 0.95,
    'U': 0.05,
})
X2.set_probabilities({
    'G': 0.98,
    'U': 0.02,
})
polycell.check_probabilities() # Make sure they sum to 1.

True

Let's see what the output is now:

In [76]:
# Run conflict-directed A*!
solution = conflict_directed_a_star(polycell_obs)
print("Solution: {}".format(solution))

# Is it the right solution?
assert_equal(solution, frozenset([A1.get_assignment('G'), A2.get_assignment('G'), A3.get_assignment('G'), X1.get_assignment('U'), X2.get_assignment('G')]))
test_ok()

[(-0.8582777279999999, frozenset())]
Popped frozenset() with p=0.8582777279999999.
 --> Adding frozenset({(A3 = U)})
 --> Adding frozenset({(A3 = G)})
Popped frozenset({(A3 = G)}) with p=0.8582777279999999.
 --> Adding frozenset({(A3 = G), (X1 = U)})
 --> Adding frozenset({(A3 = G), (X1 = G)})
Popped frozenset({(A3 = G), (X1 = G)}) with p=0.8582777279999999.
 --> Adding frozenset({(A1 = U), (A3 = G), (X1 = G)})
 --> Adding frozenset({(A3 = G), (X1 = G), (A1 = G)})
Popped frozenset({(A3 = G), (X1 = G), (A1 = G)}) with p=0.8582777279999999.
 --> Adding frozenset({(A3 = G), (X1 = G), (A2 = U), (A1 = G)})
 --> Adding frozenset({(A3 = G), (X1 = G), (A2 = G), (A1 = G)})
Popped frozenset({(A3 = G), (X1 = G), (A2 = G), (A1 = G)}) with p=0.8582777279999999.
 --> Adding frozenset({(A3 = G), (A2 = G), (X1 = G), (X2 = U), (A1 = G)})
 --> Adding frozenset({(A3 = G), (A2 = G), (X2 = G), (X1 = G), (A1 = G)})
Popped frozenset({(A3 = G), (A2 = G), (X2 = G), (X1 = G), (A1 = G)}) with p=0.858277727999999

Tests passed!!

That makese sense! Let's change the probabilities one more time, this time making a certain double fault more likely than any single fault.

In [77]:
A1.set_probabilities({
    'G': 0.99,
    'U': 0.01,
})
A2.set_probabilities({
    'G': 0.9,
    'U': 0.1,
})
A3.set_probabilities({
    'G': 0.8,
    'U': 0.2,
})
X1.set_probabilities({
    'G': 0.98,
    'U': 0.02,
})
X2.set_probabilities({
    'G': 0.97,
    'U': 0.03,
})
polycell.check_probabilities() # Make sure they sum to 1.

True

Let's see the resulting solutions:

In [78]:
# Run conflict-directed A*!
solution = conflict_directed_a_star(polycell_obs)
print("Solution: {}".format(solution))

# Is it the right solution?
assert_equal(solution, frozenset([A1.get_assignment('G'), A2.get_assignment('U'), A3.get_assignment('U'), X1.get_assignment('G'), X2.get_assignment('G')]))
test_ok()

[(-0.67758768, frozenset())]
Popped frozenset() with p=0.67758768.
 --> Adding frozenset({(A3 = U)})
 --> Adding frozenset({(A3 = G)})
Popped frozenset({(A3 = G)}) with p=0.67758768.
 --> Adding frozenset({(A3 = G), (X1 = U)})
 --> Adding frozenset({(A3 = G), (X1 = G)})
Popped frozenset({(A3 = G), (X1 = G)}) with p=0.67758768.
 --> Adding frozenset({(A1 = U), (A3 = G), (X1 = G)})
 --> Adding frozenset({(A3 = G), (X1 = G), (A1 = G)})
Popped frozenset({(A3 = G), (X1 = G), (A1 = G)}) with p=0.67758768.
 --> Adding frozenset({(A3 = G), (X1 = G), (A2 = U), (A1 = G)})
 --> Adding frozenset({(A3 = G), (X1 = G), (A2 = G), (A1 = G)})
Popped frozenset({(A3 = G), (X1 = G), (A2 = G), (A1 = G)}) with p=0.67758768.
 --> Adding frozenset({(A3 = G), (A2 = G), (X1 = G), (X2 = U), (A1 = G)})
 --> Adding frozenset({(A3 = G), (A2 = G), (X2 = G), (X1 = G), (A1 = G)})
Popped frozenset({(A3 = G), (A2 = G), (X2 = G), (X1 = G), (A1 = G)}) with p=0.67758768.
 --> Checking consistency.
 --> Nope! Got conflict fr

Tests passed!!

Makes sense! Now the most likely solution that explains our symptoms contains `A2 = U` and `A3 = U`.

## Done

That's the end of this problem set. Make sure to validate and submit!